# Universidad del Valle de Guatemala
## Minería de Datos - Hoja de Trabajo 7
### José Jorge Pérez - 18364
### Diego Ruiz - 18761

In [88]:
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
import random
import seaborn as sns
import scipy.stats as stats
from sklearn import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.svm import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.tree import *

data = pd.read_csv("train.csv")
num = data.select_dtypes(include='number')
num = num.drop(["Fireplaces", "GarageYrBlt", "Id", "MSSubClass", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "MoSold", "YrSold"], axis=1)

num = num.dropna()
# num.head()

In [89]:
percentil = np.percentile(num["SalePrice"], [33.33, 66.67])
eco_lim = percentil[0]
exp_lim = percentil[1]
num["Class"] = pd.cut(num["SalePrice"], bins=[0, eco_lim, exp_lim, float("inf")], labels=["Economic", "Medium", "Expensive"])
dumm = pd.get_dummies(num['Class'])
dumm.head()

,Economic,Medium,Expensive
0,0,0,1
1,0,1,0
2,0,0,1
3,0,1,0
4,0,0,1


In [90]:
data = pd.concat([num,dumm], axis=1)
data.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,TotRmsAbvGrd,GarageCars,GarageArea,PoolArea,MiscVal,SalePrice,Class,Economic,Medium,Expensive
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,...,8,2,548,0,0,208500,Expensive,0,0,1
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,...,6,2,460,0,0,181500,Medium,0,1,0
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,...,6,2,608,0,0,223500,Expensive,0,0,1
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,...,7,3,642,0,0,140000,Medium,0,1,0
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,...,9,3,836,0,0,250000,Expensive,0,0,1


In [91]:
econo = data.pop("Economic") 
med = data.pop("Medium")
expensive = data.pop("Expensive")
data.pop("Class")

0       Expensive
1          Medium
2       Expensive
3          Medium
4       Expensive
          ...    
1455       Medium
1456    Expensive
1457    Expensive
1458       Medium
1459       Medium
Name: Class, Length: 1195, dtype: category
Categories (3, object): ['Economic' < 'Medium' < 'Expensive']

In [92]:
from sklearn.model_selection import train_test_split
import random
X = data
y = expensive
random.seed(424)
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=424)

### *Ejercicio 2*
**Explore los datos y explique las transformaciones que debe hacerle para generar un modelo
de máquinas vectoriales de soporte.**

In [93]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

print("\n","Exact:", accuracy_score(y_test, y_pred), "\n", "Precision", precision_score(y_test, y_pred, average='weighted'),"\n","Total Points F1:", f1_score(y_test, y_pred, average='weighted'),"\n","Recall:", recall_score(y_test, y_pred, average='weighted'), "\n")



 Exact: 1.0 
 Precision 1.0 
 Total Points F1: 1.0 
 Recall: 1.0 



### *Ejercicio 4*
**Genere varios (más de 2) modelos de SVM con diferentes kernels y distintos valores en los
parámetros c, gamma (circular) y d (en caso de que utilice el polinomial). Puede tunear el
modelo de forma automática siempre que explique los resultados**

In [94]:
# param_grid = {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto']}

# svm = SVC()
# grid = GridSearchCV(svm, param_grid=param_grid, cv=5)

# grid.fit(X_train, y_train)
# print("Best Params: ", grid.best_params_)

# y_pred = grid.predict(X_test)

# print("\n","Exact:", accuracy_score(y_test, y_pred),"\n", "Precision:", precision_score(y_test, y_pred, average='weighted'),"\n", "Total Points F1:", f1_score(y_test, y_pred, average='weighted'),"\n", "Recall:", recall_score(y_test, y_pred, average='weighted') )

models = []
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
Cs = [0.1, 1, 10]
gammas = [0.1, 1, 10]
for kernel in kernels:
    for c in Cs:
        for gamma in gammas:
            models.append(SVC(kernel=kernel, C=c, gamma=gamma))

for model in models:
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print("Kernel: {}, C: {}, Gamma: {}, Accuracy: {:.2f}".format(model.kernel, model.C, model.gamma, accuracy))


Kernel: linear, C: 0.1, Gamma: 0.1, Accuracy: 1.00
Kernel: linear, C: 0.1, Gamma: 1, Accuracy: 1.00
Kernel: linear, C: 0.1, Gamma: 10, Accuracy: 1.00
Kernel: linear, C: 1, Gamma: 0.1, Accuracy: 1.00
Kernel: linear, C: 1, Gamma: 1, Accuracy: 1.00
Kernel: linear, C: 1, Gamma: 10, Accuracy: 1.00
Kernel: linear, C: 10, Gamma: 0.1, Accuracy: 1.00
Kernel: linear, C: 10, Gamma: 1, Accuracy: 1.00
Kernel: linear, C: 10, Gamma: 10, Accuracy: 1.00
Kernel: poly, C: 0.1, Gamma: 0.1, Accuracy: 1.00
Kernel: poly, C: 0.1, Gamma: 1, Accuracy: 1.00
Kernel: poly, C: 0.1, Gamma: 10, Accuracy: 1.00
Kernel: poly, C: 1, Gamma: 0.1, Accuracy: 1.00
Kernel: poly, C: 1, Gamma: 1, Accuracy: 1.00
Kernel: poly, C: 1, Gamma: 10, Accuracy: 1.00
Kernel: poly, C: 10, Gamma: 0.1, Accuracy: 1.00
Kernel: poly, C: 10, Gamma: 1, Accuracy: 1.00
Kernel: poly, C: 10, Gamma: 10, Accuracy: 1.00
Kernel: rbf, C: 0.1, Gamma: 0.1, Accuracy: 0.67
Kernel: rbf, C: 0.1, Gamma: 1, Accuracy: 0.67
Kernel: rbf, C: 0.1, Gamma: 10, Accuracy: 

### *Ejercicio 5*
**Use los modelos para predecir el valor de la variable respuesta**

In [97]:
# param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto']}

# svm = SVC()
# grid = GridSearchCV(svm, param_grid=param_grid, cv=5)

# grid.fit(X_train, y_train)

# print("Best Params: ", grid.best_params_)

# y_pred = grid.predict(X_test)

# print("\n","Exact:", accuracy_score(y_test, y_pred), "\n", "Precision:", precision_score(y_test, y_pred, average='weighted'), "\n", "Total points F1:", f1_score(y_test, y_pred, average='weighted'), "\n", "Recall:", recall_score(y_test, y_pred, average='weighted'), "\n" )

for model in models:
    model.fit(X_train, y_train)

for model in models:
    y_pred = model.predict(X_test)
    print("Kernel: {}, C: {}, Gamma: {}, Presicion Predicciones: {}".format(model.kernel, model.C, model.gamma, precision_score(y_test, y_pred, average='weighted', zero_division=0)))


Kernel: linear, C: 0.1, Gamma: 0.1, Predicciones 1.0
Kernel: linear, C: 0.1, Gamma: 1, Predicciones 1.0
Kernel: linear, C: 0.1, Gamma: 10, Predicciones 1.0
Kernel: linear, C: 1, Gamma: 0.1, Predicciones 1.0
Kernel: linear, C: 1, Gamma: 1, Predicciones 1.0
Kernel: linear, C: 1, Gamma: 10, Predicciones 1.0
Kernel: linear, C: 10, Gamma: 0.1, Predicciones 1.0
Kernel: linear, C: 10, Gamma: 1, Predicciones 1.0
Kernel: linear, C: 10, Gamma: 10, Predicciones 1.0
Kernel: poly, C: 0.1, Gamma: 0.1, Predicciones 1.0
Kernel: poly, C: 0.1, Gamma: 1, Predicciones 1.0
Kernel: poly, C: 0.1, Gamma: 10, Predicciones 1.0
Kernel: poly, C: 1, Gamma: 0.1, Predicciones 1.0
Kernel: poly, C: 1, Gamma: 1, Predicciones 1.0
Kernel: poly, C: 1, Gamma: 10, Predicciones 1.0
Kernel: poly, C: 10, Gamma: 0.1, Predicciones 1.0
Kernel: poly, C: 10, Gamma: 1, Predicciones 1.0
Kernel: poly, C: 10, Gamma: 10, Predicciones 1.0
Kernel: rbf, C: 0.1, Gamma: 0.1, Predicciones 0.44258678944696345
Kernel: rbf, C: 0.1, Gamma: 1, Pre